In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.backend as K

Using TensorFlow backend.


In [2]:
((X_train, Y_train), (X_test, Y_test)) = keras.datasets.mnist.load_data()
X_train, X_test = X_train/255, X_test/255

### a compressed matrix neural layer

In [30]:
class CompressedDense(keras.layers.Layer):
    
    def __init__(self, units, freqs=128, **kwargs):
        self.units = units
        self.freqs = freqs
        super(CompressedDense, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.inputs = input_shape[1]
        self.weight_padding = (self.inputs * self.units) - self.freqs
        self.kernel = self.add_weight(
            shape=(1,self.freqs,1),
            initializer='glorot_uniform', name='kernel')
        self.bias = self.add_weight(
            shape=(self.units,),
            initializer='zeros', name='bias')
        super(CompressedDense, self).build(input_shape)
    
    def call(self, inputs):
        weights = tf.spectral.idct(
            K.permute_dimensions(
                K.temporal_padding(
                    self.kernel, (0,self.weight_padding)), (0,2,1)), norm='ortho')
        weights = K.reshape(weights, (self.inputs, self.units))
        return inputs @ weights + self.bias
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

### baseline

In [47]:
X = X_input = keras.layers.Input((28,28))
X = keras.layers.Reshape((28*28,))(X)
X = keras.layers.Dense(10)(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('softmax')(X)
M_baseline = keras.Model(X_input, X)
M_baseline.compile('adam', 'sparse_categorical_crossentropy', ['acc'])
M_baseline.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 28, 28)            0         
_________________________________________________________________
reshape_20 (Reshape)         (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                7850      
_________________________________________________________________
batch_normalization_29 (Batc (None, 10)                40        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
Total params: 7,890
Trainable params: 7,870
Non-trainable params: 20
_________________________________________________________________


In [48]:
%%time
M_baseline.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=512, epochs=200, callbacks=[
    keras.callbacks.ReduceLROnPlateau('loss', patience=3, verbose=1),
    keras.callbacks.EarlyStopping('loss', patience=10, verbose=1)
])

Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 [==============================] - 2s 29us/step - loss: 1.0988 - acc: 0.7420 - val_loss: 0.7502 - val_acc: 0.8634
Epoch 2/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.7362 - acc: 0.8691 - val_loss: 0.6469 - val_acc: 0.8840
Epoch 3/200
60000/60000 [==============================] - 1s 15us/step - loss: 0.6503 - acc: 0.8817 - val_loss: 0.5842 - val_acc: 0.8898
Epoch 4/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5935 - acc: 0.8871 - val_loss: 0.5366 - val_acc: 0.8972
Epoch 5/200
60000/60000 [==============================] - 1s 16us/step - loss: 0.5509 - acc: 0.8924 - val_loss: 0.5039 - val_acc: 0.8976
Epoch 6/200
60000/60000 [==============================] - 1s 16us/step - loss: 0.5169 - acc: 0.8951 - val_loss: 0.4785 - val_acc: 0.8988
Epoch 7/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.4896 - acc: 0.8975 - val_loss: 0.4521 - 

60000/60000 [==============================] - 1s 14us/step - loss: 0.2557 - acc: 0.9301 - val_loss: 0.2822 - val_acc: 0.9223
Epoch 60/200
60000/60000 [==============================] - 1s 16us/step - loss: 0.2561 - acc: 0.9300 - val_loss: 0.2816 - val_acc: 0.9230
Epoch 61/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.2555 - acc: 0.9295 - val_loss: 0.2811 - val_acc: 0.9222
Epoch 62/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.2541 - acc: 0.9304 - val_loss: 0.2803 - val_acc: 0.9221
Epoch 63/200
60000/60000 [==============================] - 1s 15us/step - loss: 0.2530 - acc: 0.9305 - val_loss: 0.2808 - val_acc: 0.9244
Epoch 64/200
60000/60000 [==============================] - 1s 15us/step - loss: 0.2534 - acc: 0.9297 - val_loss: 0.2823 - val_acc: 0.9207
Epoch 65/200
60000/60000 [==============================] - 1s 13us/step - loss: 0.2530 - acc: 0.9305 - val_loss: 0.2858 - val_acc: 0.9206
Epoch 66/200
60000/60000 [==============

60000/60000 [==============================] - 1s 17us/step - loss: 0.2315 - acc: 0.9367 - val_loss: 0.2790 - val_acc: 0.9243
Epoch 117/200
60000/60000 [==============================] - 1s 15us/step - loss: 0.2305 - acc: 0.9360 - val_loss: 0.2790 - val_acc: 0.9248
Epoch 118/200
60000/60000 [==============================] - 1s 18us/step - loss: 0.2306 - acc: 0.9362 - val_loss: 0.2790 - val_acc: 0.9247
Epoch 119/200
60000/60000 [==============================] - 1s 16us/step - loss: 0.2315 - acc: 0.9353 - val_loss: 0.2790 - val_acc: 0.9248
Epoch 120/200
60000/60000 [==============================] - 1s 15us/step - loss: 0.2318 - acc: 0.9366 - val_loss: 0.2790 - val_acc: 0.9244

Epoch 00120: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.
Epoch 121/200
60000/60000 [==============================] - 1s 15us/step - loss: 0.2317 - acc: 0.9362 - val_loss: 0.2790 - val_acc: 0.9243
Epoch 122/200
60000/60000 [==============================] - 1s 15us/step - loss: 0.2329 - a

### compressed test

In [53]:
X = X_input = keras.layers.Input((28,28))
X = keras.layers.Reshape((28*28,))(X)
X = CompressedDense(10, freqs=4096)(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Activation('softmax')(X)
M = keras.Model(X_input, X)
M.compile('rmsprop', 'sparse_categorical_crossentropy', ['acc'])
M.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 28, 28)            0         
_________________________________________________________________
reshape_23 (Reshape)         (None, 784)               0         
_________________________________________________________________
compressed_dense_22 (Compres (None, 10)                4106      
_________________________________________________________________
batch_normalization_32 (Batc (None, 10)                40        
_________________________________________________________________
activation_19 (Activation)   (None, 10)                0         
Total params: 4,146
Trainable params: 4,126
Non-trainable params: 20
_________________________________________________________________


In [54]:
%%time
M.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=512, epochs=200, callbacks=[
    keras.callbacks.ReduceLROnPlateau('loss', patience=3, verbose=1),
    keras.callbacks.EarlyStopping('loss', patience=10, verbose=1)
])

Train on 60000 samples, validate on 10000 samples
Epoch 1/200
60000/60000 [==============================] - 2s 29us/step - loss: 1.4483 - acc: 0.6155 - val_loss: 1.1959 - val_acc: 0.6996
Epoch 2/200
60000/60000 [==============================] - 1s 16us/step - loss: 1.2084 - acc: 0.7087 - val_loss: 1.1052 - val_acc: 0.7155
Epoch 3/200
60000/60000 [==============================] - 1s 15us/step - loss: 1.1380 - acc: 0.7224 - val_loss: 1.0655 - val_acc: 0.7373
Epoch 4/200
60000/60000 [==============================] - 1s 16us/step - loss: 1.0789 - acc: 0.7384 - val_loss: 1.0193 - val_acc: 0.7330
Epoch 5/200
60000/60000 [==============================] - 1s 16us/step - loss: 1.0257 - acc: 0.7553 - val_loss: 0.9698 - val_acc: 0.7615
Epoch 6/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.9770 - acc: 0.7686 - val_loss: 0.9235 - val_acc: 0.7814
Epoch 7/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.9320 - acc: 0.7786 - val_loss: 0.8892 - 

60000/60000 [==============================] - 1s 14us/step - loss: 0.5350 - acc: 0.8457 - val_loss: 0.5641 - val_acc: 0.8363
Epoch 60/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5347 - acc: 0.8468 - val_loss: 0.5518 - val_acc: 0.8385
Epoch 61/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5345 - acc: 0.8473 - val_loss: 0.5541 - val_acc: 0.8385
Epoch 62/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5319 - acc: 0.8485 - val_loss: 0.5581 - val_acc: 0.8385
Epoch 63/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5322 - acc: 0.8476 - val_loss: 0.5505 - val_acc: 0.8394
Epoch 64/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5314 - acc: 0.8478 - val_loss: 0.5485 - val_acc: 0.8422
Epoch 65/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5303 - acc: 0.8478 - val_loss: 0.5563 - val_acc: 0.8357
Epoch 66/200
60000/60000 [==============


Epoch 00114: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-09.
Epoch 115/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5059 - acc: 0.8552 - val_loss: 0.5383 - val_acc: 0.8467
Epoch 116/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5061 - acc: 0.8555 - val_loss: 0.5383 - val_acc: 0.8465
Epoch 117/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5046 - acc: 0.8551 - val_loss: 0.5383 - val_acc: 0.8463

Epoch 00117: ReduceLROnPlateau reducing learning rate to 1.000000082740371e-10.
Epoch 118/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5058 - acc: 0.8550 - val_loss: 0.5383 - val_acc: 0.8466
Epoch 119/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5044 - acc: 0.8558 - val_loss: 0.5382 - val_acc: 0.8466
Epoch 120/200
60000/60000 [==============================] - 1s 14us/step - loss: 0.5050 - acc: 0.8560 - val_loss: 0.5382 - val_acc: 0.846